## Azure AI Search
### Using an AI Search Index for context data

In [ ]:
# Set up the query for generating responses
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential

from azure.identity import get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)

credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
# token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

openai_client = AzureOpenAI(
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

search_client = SearchClient(
    endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    index_name=os.getenv("AZURE_SEARCH_INDEX_NAME"),
    credential=credential
)

result = search_client.get_document_count()

# This prompt provides instructions to the model
GROUNDED_PROMPT="""
You are a very smart AI researcher helping users find answers to their questions. Your answers are short and simple.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# Query is the question being asked. It's sent to the search engine and the chat model
query="Cache Fusion"

# Search results are created by the search client
# Search results are composed of the top 5 results and the fields selected from the search index
# Search results include the top 5 matches to your query
search_results = search_client.search(
    search_text=query,
    top=5,
    select="title, chunk",
    vector_queries=[{
        "kind": "text",
        "text": query,
        "fields": "text_vector",
        "k": 3,
    }],
)

sources_formatted = "\n\n".join([f'{document["title"]}:{document["chunk"]}' for document in search_results])

print(sources_formatted)

### Simple Q&A Test (no evaluation)

In [ ]:
# Send the search results and the query to the LLM to generate a response based on the prompt.
response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT")
)

# Here is the response from the chat model.
print(response.choices[0].message.content)

### Setup Evaluation Logic

In [ ]:
from typing import List, Dict, Any, Optional
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider


def call_to_your_ai_application(query: str) -> str:
    # logic to call your application
    # use a try except block to catch any errors

    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        # azure_ad_token_provider=token_provider,
    )
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )
    
    message = completion.to_dict()["choices"][0]["message"]
    return message["content"]


async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = None
    # call your endpoint or ai application here
    response = call_to_your_ai_application(query)
    # we are formatting the response to follow the openAI chat protocol format
    formatted_response = {
        "content": response,
        "role": "assistant",
        "context": {
            "citations": None,
        },
    }
    messages["messages"].append(formatted_response)
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}

### Run Simulator to synthesize Q&A pairs from search query and index context

In [ ]:
from azure.ai.evaluation.simulator import Simulator

simulator = Simulator(model_config=model_config)

outputs = await simulator(
    target=callback,
    text=sources_formatted,
    num_queries=4,
    max_conversation_turns=1,
    tasks=[
        f"I want to learn more about {query}",
    ],
)

In [ ]:
from pathlib import Path

output_file = Path("output.json")
with output_file.open("a") as f:
    json.dump(outputs, f)

### Running evaluations on the simulated data
Here we will try to run GroundednessEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator on the output data from the simulator.

From the documentation we know that running those evaluators need the following data: `query`, `response`, `context`, `ground_truth`

For simplicity's sake, we can use our source document `text` as both `context` and `ground_truth`. This step only evaluates the first user message and first response from your AI Application for each of the simulated conversations.

In [ ]:
eval_input_data_json_lines = ""
for output in outputs:
    query = None
    response = None
    context = sources_formatted
    ground_truth = sources_formatted
    for message in output["messages"]:
        if message["role"] == "user":
            query = message["content"]
        if message["role"] == "assistant":
            response = message["content"]
    if query and response:
        eval_input_data_json_lines += (
            json.dumps(
                {
                    "query": query,
                    "response": response,
                    "context": context,
                    "ground_truth": ground_truth,
                }
            )
            + "\n"
        )

Store the output in a file

In [ ]:
eval_input_data_file = Path("eval_input_data.jsonl")

with eval_input_data_file.open("w") as f:
    f.write(eval_input_data_json_lines)

### Run evaluation
`QAEvaluator` is a composite evaluator which runs GroundednessEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator

Optionally set the azure_ai_project to upload the evaluation results to Azure AI Studio.

In [ ]:
from azure.ai.evaluation import evaluate, QAEvaluator

qa_evaluator = QAEvaluator(model_config=model_config)

eval_output = evaluate(
    data=str(eval_input_data_file),
    evaluators={"QAEvaluator": qa_evaluator},
    evaluator_config={
        "QAEvaluator": {
            "column_mapping": {
                "query": "${data.query}",
                "response": "${data.response}",
                "context": "${data.context}",
                "ground_truth": "${data.ground_truth}",
            }
        }
    },
    azure_ai_project=azure_ai_project,  # optional to store the evaluation results in Azure AI Studio
    output_path="./myevalresults.json",  # optional to store the evaluation results in a file
)